# Extracting information from input in structured manner

This notebook performs simple extraction of information about a coffee order.

## Setup

The usual boiler plate to import libraries and set the model.

You'll need your OpenAI key set as environment variable `OPENAI_API_KEY`

In [1]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from open_ai_functions import pretty_print_conversation,chat_completion_request

GPT_MODEL = "gpt-4"

## Function  definitions
Next, we define the function arguments - the structured data we want to extract from the free text.

In [2]:
## We could send this to an automated coffee machine
def coffee (coffee_order):
    print(f'Ordering coffee: {json.dumps(coffee_order, indent=2)}')

## a dictionary of functions keyed by their name. 
available_functions = {
    "coffee":coffee, 
}

## The function definitions we will send to ChatGPT. The 'parameters' object is defined using JSON Schema.
functions = [
    {
        "name" : "coffee",
        "description": " Get the coffee order from the input ",
        "parameters" : {
            "type": "object",
            "properties" : {
                  "order": {
                      "type":"object",
                      "properties" : {
                        "coffee_type": {
                         "type":"string"
                        }, 
                        "temperature": {
                          "type":"string"
                        },
                        "size": {
                          "type":"string"
                        },
                        "milks": {
                          "type":"string"
                        },
                        "sugar": {
                          "type":"boolean",
                          "enum": ["true", "false"]
                        },
                        "sugar_count": {
                          "type":"number"
                        },
                        "syrups": {
                          "type":"string"
                        }
                    }         
                }
            }
        }
    }
]
        

## Executing the conversation

In [6]:
inputs = [
    "I'd like a tall cold latte please with two sugars and hazelnut syrup",
    "What's the weather in London"
]

In [9]:

for input in inputs:
    messages = [
      {
     "role":"system",
     "content":"Extract user input into structured data"
      }
    ]
    messages.append({"role":"user", "content" : input})
    resp = chat_completion_request(messages, functions=functions)

    response_message = resp.json()['choices'][0]['message']
    
    messages.append(response_message)
    pretty_print_conversation(messages)
    
    if 'function_call' in response_message and response_message['function_call'] is not None:
        to_call = response_message['function_call']['name']
        f_args = json.loads(response_message['function_call']['arguments'])['order']
        result = available_functions[to_call](f_args)

    else:
        print(f'no function was returned')
    print ('-----------------------------------')
    print ('-----------------------------------')
    

system: Extract user input into structured data

user: I'd like a tall cold latte please with two sugars and hazelnut syrup

assistant: {'name': 'coffee', 'arguments': '{\n  "order": {\n    "coffee_type": "latte",\n    "temperature": "cold",\n    "size": "tall",\n    "sugar": true,\n    "sugar_count": 2,\n    "syrups": "hazelnut"\n  }\n}'}

Ordering coffee: {
  "coffee_type": "latte",
  "temperature": "cold",
  "size": "tall",
  "sugar": true,
  "sugar_count": 2,
  "syrups": "hazelnut"
}
-----------------------------------
-----------------------------------
system: Extract user input into structured data

user: What's the weather in London

assistant: This prompt doesn't match the function "functions.coffee". Therefore, no extraction can be made.

no function was returned
-----------------------------------
-----------------------------------
